In [4]:
from selenium import webdriver
from time import sleep
import json
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup

In [83]:

# 나중에 이거 주소를 리스트에 넣고 하나씩 반복해서 돌리면 될거같음
# 이거 저기 ms부분만 리스트에 넣고 url주소 바꾸면서 가져오면 될거같음 (ms가 위도경도임) ->이거안됨
# 용산구 주성동, 영등포구 양평동, 용상구 주성동, 종로구 교남동, 종로구 권농동, 종로구 종로 1가,2가, 종로구 필운동, 중구 소공동 중구 의지로1가
# 중구 필동 2가 3가
giyeok_list = ['37.576192,126.921263', '37.5672,126.9628']
total_result = []
def crawling(data):
    driver = webdriver.Chrome('./chromedriver')
    for item in data:
        zuso = 'https://new.land.naver.com/rooms?ms='+item+',11&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ae=ONEROOM'
        driver.get(zuso)
        sleep(2)

        content = BeautifulSoup(driver.page_source, 'html.parser')
        driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
        
        # 원룸/투룸 클릭 
        driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
        sleep(1)

        # 혹시 리스트가 갱신이 안되는 경우를 제외하기위해 새로고침 버튼 클릭
        driver.find_element_by_css_selector("#filter > button > i ").click()  
        sleep(1)
        
        # 투룸 제외(원룸만) -> 이거 체크되어있는지 확인하고(area-pressed = true인지 false인지)
        # 체크되어있으면 클릭 안하도록 만들기
        content = BeautifulSoup(driver.page_source, 'html.parser')
        tworoom = content.find('button',{'aria-pressed':'false'}).text
        if tworoom != " 투룸":
            driver.find_element_by_css_selector("#type1").click()
            sleep(1)
             
        
        driver.find_element_by_css_selector("#list > div > div > div.list_fixed > div > a:nth-child(2)").click()
        sleep(1)

        actions = ActionChains(driver)

        # 페이지 전체를 스크롤다운 하려는게 아니라 옆에 리스트만 스크롤 다운하고 싶기 때문에 이걸 해줌
        scrollArea = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
        sleep(1)

        # driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)

        # actions.move_to_element(element).perform()
        last_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)

        try:
            a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
        except NoSuchElementException:
            a = None

        while(True):
            # 무한스크롤의 끝까지 늘려줌
            # 스크롤이 끝나기 전에는 loader이라는 div가 있는데 그게 더 이상 없을 때까지 스크롤을 내려줌  
            while a != None:
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)
                sleep(1)
                try:
                    new_a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
                except NoSuchElementException:
                    
                    new_a = None
                sleep(1)    
                if new_a != a:
                    a = new_a

            new_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)
            if new_height == last_height:
                break
            last_height = new_height

        print("loading complete")
        # new_height = driver.execute_script("arguments[0].scrollTo(0, 0);", scrollArea)
        # 스크롤바 제일 위로 올라감
        driver.execute_script("arguments[0].scrollTo(0,0);", scrollArea)
        sleep(1)

        content = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 건물명
        names = []
        list_name = content.select('#listContents1 > div > div > div > div > div > a.item_link > div.item_title > span.text')
        for item in list_name:
            names.append(item.text)

        num = len(names)
        print(num)

        # 도시명
        city = []
        Gu = []
        Dong = []
        place = []
        list_area = content.select('span.area.is-selected')
        for item in list_area:
            place.append(item.text)
        city.append(place[0])
        Gu.append(place[1])
        Dong.append(place[2])

        # 월세/전세
        types = []
        pp_type = content.select('span.text')
        for item in pp_type:
            types.append(item.text)

        # 가격
        prices = []
        pp_price = content.select('span.price')
        for item in pp_price:
            prices.append(item.text)

        # 건물 형태
        buildings = []
        pp_build = content.select('strong.type')
        for item in pp_build:
            buildings.append(item.text)

        # 기타정보
        p_spec = []
        specs=[]
        pp_spec = content.select('span.spec')
        for item in pp_spec:
            p_spec.append(item.text)
        i = 0
        while i < len(p_spec):
            if i % 2 == 0:
                specs.append(p_spec[i])
            i += 1

        # 판매여부
        checks = []
        pp_check = content.select('em.title')
        for item in pp_check:
            checks.append(item.text)

        # 날짜
        dates = []
        pp_date = content.select('em.data')
        for item in pp_date:
            dates.append(item.text)


        for i in range(len(names)):
            h = []
            h.append(city[0])
            h.append(Gu[0])
            h.append(Dong[0])
            h.append(names[i])
            h.append(types[i])
            h.append(prices[i])
            h.append(buildings[i])
            h.append(specs[i])
            h.append(checks[i])
            h.append(dates[i])
            total_result.append(h)

    data = pd.DataFrame(total_result, columns = ['시', '구', '동', '이름', '월세/전세', '가격', '건물형태', '기타정보', '확인/판매여부', '날짜'])
    data.drop_duplicates(inplace = True)
    data.to_csv('testCrawl_1.csv', encoding='cp949')


In [84]:
crawling(giyeok_list)

loading complete
71
loading complete
4


In [2]:
giyeok_list = ['37.576192,126.921263', '37.5672,126.9628']
total_result = []

In [81]:
driver = webdriver.Chrome('./chromedriver')
zuso = 'https://new.land.naver.com/rooms?ms=37.559527,126.9435,11&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ae=ONEROOM'
driver.get(zuso)
sleep(2)

content = BeautifulSoup(driver.page_source, 'html.parser')
driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
        
# 원룸/투룸 클릭 
driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
sleep(1)

# 혹시 리스트가 갱신이 안되는 경우를 제외하기위해 새로고침 버튼 클릭
driver.find_element_by_css_selector("#filter > button > i ").click()  
sleep(1)
        
# 투룸 제외(원룸만) -> 이거 체크되어있는지 확인하고(area-pressed = true인지 false인지)
# 체크되어있으면 클릭 안하도록 만들기
content = BeautifulSoup(driver.page_source, 'html.parser')
tworoom = content.find('button',{'aria-pressed':'false'}).text
if tworoom != " 투룸":
    driver.find_element_by_css_selector("#type1").click()
    sleep(1)
             
        
driver.find_element_by_css_selector("#list > div > div > div.list_fixed > div > a:nth-child(2)").click()
sleep(1)

actions = ActionChains(driver)

# 페이지 전체를 스크롤다운 하려는게 아니라 옆에 리스트만 스크롤 다운하고 싶기 때문에 이걸 해줌
scrollArea = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
sleep(1)

# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)

# actions.move_to_element(element).perform()
last_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)

try:
    a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
except NoSuchElementException:
    a = None

while(True):
    # 무한스크롤의 끝까지 늘려줌
    # 스크롤이 끝나기 전에는 loader이라는 div가 있는데 그게 더 이상 없을 때까지 스크롤을 내려줌  
    while a != None:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)
        sleep(1)
        try:
            new_a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
        except NoSuchElementException:
                    
            new_a = None
        sleep(1)    
        if new_a != a:
            a = new_a

    new_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)
    if new_height == last_height:
        break
    last_height = new_height

print("loading complete")
# new_height = driver.execute_script("arguments[0].scrollTo(0, 0);", scrollArea)
# 스크롤바 제일 위로 올라감
driver.execute_script("arguments[0].scrollTo(0,0);", scrollArea)
sleep(1)

content = BeautifulSoup(driver.page_source, 'html.parser')
        
class House:
    def __init__(self,p_city,p_Gu,p_Dong,p_name,p_type,p_price,p_build,p_spec,p_check,p_date):
        self.p_city = p_city
        self.p_Gu = p_Gu
        self.p_Dong = p_Dong
        self.p_name = p_name
        self.p_type = p_type
        self.p_price = p_price
        self.p_build = p_build
        self.p_spec = p_spec
        self.p_check = p_check
        self.p_date = p_date
            
    def __str__(self):
        return self.p_city +', '+ self.p_Gu +', '+ self.p_Dong +', '+ self.p_name +', '+ self.p_type +', '+ self.p_price +', '+ self.p_build +', '+ self.p_spec +', '+ self.p_check +', '+ self.p_date
         
    def __repr__(self):
        return str(self)
        
# 건물명
names = []
list_name = content.select('#listContents1 > div > div > div > div > div > a.item_link > div.item_title > span.text')
for item in list_name:
    names.append(item.text)

num = len(names)
print(num)
        
# 도시명
city = []
Gu = []
Dong = []
place = []
list_area = content.select('span.area.is-selected')
for item in list_area:
    place.append(item.text)
city.append(place[0])
Gu.append(place[1])
Dong.append(place[2])
        
# 월세/전세
types = []
pp_type = content.select('span.text')
for item in pp_type:
    types.append(item.text)
        
# 가격
prices = []
pp_price = content.select('span.price')
for item in pp_price:
    prices.append(item.text)
    
# 건물 형태
buildings = []
pp_build = content.select('strong.type')
for item in pp_build:
    buildings.append(item.text)
    
# 기타정보
p_spec = []
specs=[]
pp_spec = content.select('span.spec')
for item in pp_spec:
    p_spec.append(item.text)
i = 0
while i < len(p_spec):
    if i % 2 == 0:
        specs.append(p_spec[i])
    i += 1
    
# 판매여부
checks = []
pp_check = content.select('em.title')
for item in pp_check:
    checks.append(item.text)
    
# 날짜
dates = []
pp_date = content.select('em.data')
for item in pp_date:
    dates.append(item.text)


loading complete
292


In [7]:
print(types)

['일반원룸 ', '일반원룸 성산빌딩 ', '일반원룸 성산빌딩 ', '일반원룸 ', '일반지도 범례', '지적편집도 범례']


In [8]:
print(prices)

['1,000/40', '1,000/100', '1,000/100', '300/30']


In [9]:
print(buildings)

['원룸', '원룸', '원룸', '원룸']


In [ ]:
# 기타정보
p_spec = []
specs=[]
pp_spec = content.select('span.spec')
for item in pp_spec:
    p_spec.append(item.text)
    
i = 0
while i < len(p_spec):
    if i % 2 == 0:
        specs.append(p_spec[i])
    i += 1
    

In [26]:
print(specs)
print(len(specs))

['15/15m², 3/3층', '102/40m², 6/8층', '102/40m², 7/7층', '15/14m², 2/2층, 남향']
4


In [10]:
print(checks)

['확인', '확인', '확인', '등록일']


In [11]:
print(dates)

['19.11.27.', '19.11.22.', '19.11.18.', '19.11.18.']


In [12]:
print(types)

['일반원룸 ', '일반원룸 성산빌딩 ', '일반원룸 성산빌딩 ', '일반원룸 ', '일반지도 범례', '지적편집도 범례']


In [13]:
print(names)

['일반원룸 ', '일반원룸 성산빌딩 ', '일반원룸 성산빌딩 ', '일반원룸 ']


In [82]:
total_result = []

for i in range(len(names)):
    h = []
    h.append(city[0])
    h.append(Gu[0])
    h.append(Dong[0])
    h.append(names[i])
    h.append(types[i])
    h.append(prices[i])
    h.append(buildings[i])
    h.append(specs[i])
    h.append(checks[i])
    h.append(dates[i])
    total_result.append(h)

print(total_result)

[['서울시', '서대문구', '대현동', '일반원룸 ', '일반원룸 ', '8,000', '원룸', '27/26m², 3/5층', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '이대프리젠#1 1동', '이대프리젠#1 1동', '1,000/65', '오피스텔', '23B/14m², 7/10층, 남향', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '일반원룸 ', '일반원룸 ', '1,000/90', '원룸', '29/29m², 8/9층, 남향', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '신촌자이엘라 1동', '신촌자이엘라 1동', '1,000/85', '오피스텔', '58B1/28m², 8/15층, 남서향', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '오피스텔 ', '오피스텔 ', '1,000/95', '원룸', '40/36m², 11/15층', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '일반원룸 ', '일반원룸 ', '1,000/95', '원룸', '12/9m², 11/11층, 북향', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '일반원룸 이화애 하우스 ', '일반원룸 이화애 하우스 ', '1,000/60', '원룸', '25/23m², 3/5층', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '이대더가온 1동', '이대더가온 1동', '4,000/55', '오피스텔', '26H/16m², 9/10층, 남향', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '빌라 ', '빌라 ', '120/120', '빌라', '28/22m², 8/15층', '확인', '19.12.04.'], ['서울시', '서대문구', '대현동', '이대역마에스트로(도시형) 1동', '이대역마에스트로(도시형) 1동', '2

In [73]:
d =[]
for item in total_result:
    for i in item:
        d.append(i.split(","))

AttributeError: 'House' object has no attribute 'split'

In [77]:
data = pd.DataFrame(total_result)

data

,0,1,2,3,4,5,6,7,8,9
0,서울시,서대문구,냉천동,일반원룸,일반원룸,"1,000/40",원룸,"15/15m², 3/3층",확인,19.11.27.
1,서울시,서대문구,냉천동,일반원룸 성산빌딩,일반원룸 성산빌딩,"1,000/100",원룸,"102/40m², 6/8층",확인,19.11.22.
2,서울시,서대문구,냉천동,일반원룸 성산빌딩,일반원룸 성산빌딩,"1,000/100",원룸,"102/40m², 7/7층",확인,19.11.18.
3,서울시,서대문구,냉천동,일반원룸,일반원룸,300/30,원룸,"15/14m², 2/2층, 남향",등록일,19.11.18.


In [67]:
class House:
    def __init__(self,p_city,p_Gu,p_Dong,p_name,p_type,p_price,p_build,p_spec,p_check,p_date):
        self.p_city = p_city
        self.p_Gu = p_Gu
        self.p_Dong = p_Dong
        self.p_name = p_name
        self.p_type = p_type
        self.p_price = p_price
        self.p_build = p_build
        self.p_spec = p_spec
        self.p_check = p_check
        self.p_date = p_date
            
    def __str__(self):
        return  self.p_city +', '+ self.p_Gu +', '+ self.p_Dong +', '+ self.p_name +', '+ self.p_type +', '+ self.p_price +', '+ self.p_build +', '+ self.p_spec +', '+ self.p_check +', '+ self.p_date
    
    def __repr__(self):
        return str(self)

In [ ]:
String str = "홍길동"!"